In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import keras


CIFAR_DIR='/home/csunix/ml17z2x/Downloads/practice/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

Using TensorFlow backend.


['readme.html', 'data_batch_2', 'batches.meta', 'data_batch_1', 'data_batch_3', 'data_batch_4', 'test_batch', 'data_batch_5']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']


In [3]:
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        #print(all_data)            
        self._data = np.vstack(all_data)
        #print(self._data)
        
        self._data = self._data / 255
        #print(self._data)
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        
        #print(self._data.shape)
        #print(self._labels.shape)
        if self._need_shuffle:
            self._shuffle_data()
    
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        #print(self._indicator)
        #print(batch_size)
        if end_indicator > self._num_examples:
            #print(end_indicator)
            #print(self._num_examples)
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

#print(type(train_filenames))
#print(train_filenames)
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(100)
#print(batch_data)
#print(batch_labels)

In [4]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 3072]) #placeholder 占位图  ,None 表示样本数不确定
y = tf.placeholder(tf.int64,[None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm = [0, 3, 2, 1])

#(32*32)
conv1 = tf.layers.conv2d(x_image,
                        32,
                        (3,3),
                        padding = 'same',
                        activation = tf.nn.relu,
                        name = 'conv1')

pooling1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), name = 'pool1')


#(16*16)
conv2 = tf.layers.conv2d(pooling1,
                        32,
                        (3,3),
                        padding = 'same',
                        activation = tf.nn.relu,
                        name = 'conv2')

pooling2 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), name = 'pool2')


#(8*8)
conv3 = tf.layers.conv2d(pooling2,
                        32,
                        (3,3),
                        padding = 'same',
                        activation = tf.nn.relu,
                        name = 'conv3')

pooling3 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), name = 'pool3')


#(4*4)


#(4*4*32)matrix


flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels= y, logits= y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))



with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)



In [6]:


init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100


with tf.Session() as sess:
    
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={x: batch_data, y: batch_labels})
        
        if (i+1) % 500 == 0:
            print('[Train]Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
            
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test]Step: %d, acc: %4.5f' % (i+1,test_acc))

[Train]Step: 500, loss: 1.20942, acc: 0.55000
[Train]Step: 1000, loss: 1.04560, acc: 0.60000
[Train]Step: 1500, loss: 1.77555, acc: 0.35000
[Train]Step: 2000, loss: 1.37530, acc: 0.50000
[Train]Step: 2500, loss: 1.79461, acc: 0.35000
[Train]Step: 3000, loss: 1.17900, acc: 0.60000
[Train]Step: 3500, loss: 0.72430, acc: 0.65000
[Train]Step: 4000, loss: 1.11408, acc: 0.70000
[Train]Step: 4500, loss: 0.94528, acc: 0.65000
[Train]Step: 5000, loss: 0.69562, acc: 0.80000
[Test]Step: 5000, acc: 0.64150
[Train]Step: 5500, loss: 1.16578, acc: 0.60000
[Train]Step: 6000, loss: 0.93627, acc: 0.65000
[Train]Step: 6500, loss: 0.92420, acc: 0.70000
[Train]Step: 7000, loss: 0.81798, acc: 0.70000
[Train]Step: 7500, loss: 0.69958, acc: 0.80000
[Train]Step: 8000, loss: 0.98147, acc: 0.60000
[Train]Step: 8500, loss: 0.95677, acc: 0.70000
[Train]Step: 9000, loss: 0.92493, acc: 0.70000
[Train]Step: 9500, loss: 1.15177, acc: 0.55000
[Train]Step: 10000, loss: 0.60966, acc: 0.75000
[Test]Step: 10000, acc: 0.685

[Train]Step: 81000, loss: 0.51123, acc: 0.70000
[Train]Step: 81500, loss: 0.45867, acc: 0.85000
[Train]Step: 82000, loss: 0.45663, acc: 0.90000
[Train]Step: 82500, loss: 0.47183, acc: 0.85000
[Train]Step: 83000, loss: 0.74692, acc: 0.75000
[Train]Step: 83500, loss: 0.33047, acc: 0.85000
[Train]Step: 84000, loss: 0.79065, acc: 0.70000
[Train]Step: 84500, loss: 0.53250, acc: 0.85000
[Train]Step: 85000, loss: 0.35124, acc: 0.95000
[Test]Step: 85000, acc: 0.69600
[Train]Step: 85500, loss: 0.39297, acc: 0.90000
[Train]Step: 86000, loss: 0.23275, acc: 0.90000
[Train]Step: 86500, loss: 0.51257, acc: 0.80000
[Train]Step: 87000, loss: 0.65117, acc: 0.80000
[Train]Step: 87500, loss: 0.40340, acc: 0.85000
[Train]Step: 88000, loss: 0.54916, acc: 0.85000
[Train]Step: 88500, loss: 0.51908, acc: 0.90000
[Train]Step: 89000, loss: 0.40071, acc: 0.80000
[Train]Step: 89500, loss: 0.26788, acc: 0.95000
[Train]Step: 90000, loss: 0.59706, acc: 0.80000
[Test]Step: 90000, acc: 0.69150
[Train]Step: 90500, loss